In [1]:
# load libraies

%run py_libraries.py

/Users/4476224/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.13.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# loading utility files

from utility.sv_fig import savefig
from utility.make_cm import make_confusion_matrix
from utility.get_g_result import get_gamma_results
from utility.plt_result import plot_results

In [3]:
def savefig(filename, crop = True):
    plt.savefig('{}.pdf'.format(filename))

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

# load data
X_train = pd.read_csv('data/norm_X_train_NCa_PCa.csv')
X_test = pd.read_csv('data/norm_X_test_NCa_PCa.csv')
y_train = pd.read_csv('data/y_train_NCa_PCa.csv')
y_test = pd.read_csv('data/y_test_NCa_PCa.csv')


print(X_train.shape)

(56, 37)


In [5]:
# Print the column names as a list
print(X_train.columns.tolist())

['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']


In [6]:
Xy_NCa_PCa = pd.concat([X_train, y_train], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy import stats

col_to_scale = ['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 
                'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 
                'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG' ]

def midas_impl(missing, col_to_scale, n_epoch):
    missing_cp = missing.copy() # make copy of missing df
    
    # scaler = MinMaxScaler().fit(missing_cp[col_to_scale]) # scale data
    scaler = StandardScaler().fit(missing_cp[col_to_scale]) # scale data
    missing_sc = scaler.transform(missing_cp[col_to_scale]) # this is an array, convert to df

    missing_df = pd.DataFrame({'ENA.78':missing_sc[:,0],
                               'IFN.y':missing_sc[:,1],
                               'IL.10':missing_sc[:,2],
                               'IL.6':missing_sc[:,3],
                               'IL.8':missing_sc[:,4],
                               'MCP.1':missing_sc[:,5],
                               'MDC':missing_sc[:,6],
                               'MIP.1a':missing_sc[:,7],
                               'TNF.a':missing_sc[:,8],
                               'C.peptide':missing_sc[:,9],
                               'G.CSF':missing_sc[:,10],
                               'IL.22':missing_sc[:,11],
                               'Insulin':missing_sc[:,12],
                               'Leptin':missing_sc[:,13],
                               'MIP.3a':missing_sc[:,14],
                               'GRO.a':missing_sc[:,15],
                               'HGF':missing_sc[:,16],
                               'MMP.2':missing_sc[:,17],
                               'Adiponectin':missing_sc[:,18],
                               'CRP':missing_sc[:,19],
                               'GDF.15':missing_sc[:,20],
                               'TIMP.1':missing_sc[:,21],
                               'TGF.B2':missing_sc[:,22],
                               'TGF.B1':missing_sc[:,23],
                               'PPAR.y':missing_sc[:,24],
                               'HIF.1a':missing_sc[:,25],
                               'Laminin':missing_sc[:,26],
                               'HbA1c':missing_sc[:,27],
                               'CA19.9':missing_sc[:,28],
                               'Glucose':missing_sc[:,29],
                               'HDL':missing_sc[:,30],
                               'CCK':missing_sc[:,31],
                               'LDL':missing_sc[:,32],
                               'Triglyceride':missing_sc[:,33],
                               'Albumin':missing_sc[:,34],
                               'Lumican':missing_sc[:,35],
                               'ZAG':missing_sc[:,36],
                               })
    
    # Extract cat columns from missing
    ext_col1 = missing_cp['CACHEXSTAGE0VIG']
    

    # Add the extracted cat columns to missing_df
    missing_df = pd.concat([missing_df, ext_col1.rename('CACHEXSTAGE0VIG')], axis=1)
    

    missing_df.to_csv('missing_NCa_PCa.csv')
    data_0 = pd.read_csv('missing_NCa_PCa.csv')
    data_0.columns.str.strip()

    categorical = ['CACHEXSTAGE0VIG']
    
    data_cat, cat_cols_list = md.cat_conv(data_0[categorical])

    data_0.drop(categorical, axis = 1, inplace = True)
    constructor_list = [data_0]
    constructor_list.append(data_cat)
    data_in = pd.concat(constructor_list, axis=1)

    na_loc = data_in.isnull()
    data_in[na_loc] = np.nan

    # imputer = md.Midas(layer_structure = [128,128], vae_layer = True, seed = 89, input_drop = 0.75, learn_rate = 0.0001)
    imputer = md.Midas(layer_structure = [128,128], vae_layer = False, seed = 42, input_drop = 0.75, learn_rate = 0.001)
    imputer.build_model(data_in, softmax_columns = cat_cols_list)
    imputer.train_model(training_epochs = n_epoch)

    # do not update M
    imputations = imputer.generate_samples(m=20).output_list

    flat_cats = [cat for variable in cat_cols_list for cat in variable]
    for i in range(len(imputations)):
        tmp_cat = [imputations[i][x].idxmax(axis=1) for x in cat_cols_list]
        cat_df = pd.DataFrame({categorical[i]:tmp_cat[i] for i in range(len(categorical))})
        imputations[i] = pd.concat([imputations[i], cat_df], axis = 1).drop(flat_cats, axis = 1)

    mapping_ca = {'CACHEXSTAGE0VIG_0.0': 0,'CACHEXSTAGE0VIG_1.0': 1}

    imputations[0] = imputations[0].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[1] = imputations[1].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[2] = imputations[2].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[3] = imputations[3].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[4] = imputations[4].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[5] = imputations[5].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[6] = imputations[6].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[7] = imputations[7].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[8] = imputations[8].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[9] = imputations[9].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[10] = imputations[10].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[11] = imputations[11].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[12] = imputations[12].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[13] = imputations[13].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[14] = imputations[14].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[15] = imputations[15].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[16] = imputations[16].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[17] = imputations[17].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[18] = imputations[18].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[19] = imputations[19].replace({'CACHEXSTAGE0VIG': mapping_ca})

    # rescale imputations
    imputations[0][col_to_scale] = scaler.inverse_transform(imputations[0][col_to_scale])
    imputations[1][col_to_scale] = scaler.inverse_transform(imputations[1][col_to_scale])
    imputations[2][col_to_scale] = scaler.inverse_transform(imputations[2][col_to_scale])
    imputations[3][col_to_scale] = scaler.inverse_transform(imputations[3][col_to_scale])
    imputations[4][col_to_scale] = scaler.inverse_transform(imputations[4][col_to_scale])
    imputations[5][col_to_scale] = scaler.inverse_transform(imputations[5][col_to_scale])
    imputations[6][col_to_scale] = scaler.inverse_transform(imputations[6][col_to_scale])
    imputations[7][col_to_scale] = scaler.inverse_transform(imputations[7][col_to_scale])
    imputations[8][col_to_scale] = scaler.inverse_transform(imputations[8][col_to_scale])
    imputations[9][col_to_scale] = scaler.inverse_transform(imputations[9][col_to_scale])
    imputations[10][col_to_scale] = scaler.inverse_transform(imputations[10][col_to_scale])
    imputations[11][col_to_scale] = scaler.inverse_transform(imputations[11][col_to_scale])
    imputations[12][col_to_scale] = scaler.inverse_transform(imputations[12][col_to_scale])
    imputations[13][col_to_scale] = scaler.inverse_transform(imputations[13][col_to_scale])
    imputations[14][col_to_scale] = scaler.inverse_transform(imputations[14][col_to_scale])
    imputations[15][col_to_scale] = scaler.inverse_transform(imputations[15][col_to_scale])
    imputations[16][col_to_scale] = scaler.inverse_transform(imputations[16][col_to_scale])
    imputations[17][col_to_scale] = scaler.inverse_transform(imputations[17][col_to_scale])
    imputations[18][col_to_scale] = scaler.inverse_transform(imputations[18][col_to_scale])
    imputations[19][col_to_scale] = scaler.inverse_transform(imputations[19][col_to_scale])

    n=0
    for i in imputations:
        file_out_csv = "midas_NCa_PCa/midas_NCa_PCa_" + str(n) + ".csv"
        i.to_csv(file_out_csv, index=False)
        n += 1

    df0 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_0.csv'); df0 = df0.iloc[:,1:]
    df1 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_1.csv'); df1 = df1.iloc[:,1:]
    df2 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_2.csv'); df2 = df2.iloc[:,1:]
    df3 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_3.csv'); df3 = df3.iloc[:,1:]
    df4 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_4.csv'); df4 = df4.iloc[:,1:]
    df5 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_5.csv'); df5 = df5.iloc[:,1:]
    df6 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_6.csv'); df6 = df6.iloc[:,1:]
    df7 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_7.csv'); df7 = df7.iloc[:,1:]
    df8 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_8.csv'); df8 = df8.iloc[:,1:]
    df9 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_9.csv'); df9 = df9.iloc[:,1:]
    df10 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_10.csv'); df10 = df10.iloc[:,1:]
    df11 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_11.csv'); df11 = df11.iloc[:,1:]
    df12 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_12.csv'); df12 = df12.iloc[:,1:]
    df13 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_13.csv'); df13 = df13.iloc[:,1:]
    df14 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_14.csv'); df14 = df14.iloc[:,1:]
    df15 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_15.csv'); df15 = df15.iloc[:,1:]
    df16 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_16.csv'); df16 = df16.iloc[:,1:]
    df17 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_17.csv'); df17 = df17.iloc[:,1:]
    df18 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_18.csv'); df18 = df18.iloc[:,1:]
    df19 = pd.read_csv('midas_NCa_PCa/midas_NCa_PCa_19.csv'); df19 = df19.iloc[:,1:]

    # Initialize the output dataframe with the same dimensions
    output_df = pd.DataFrame(index=df0.index, columns=df0.columns)

    # Process continuous columns
    for col in df1.columns[:35]:
        output_df[col] = (abs(df0[col]) + abs(df1[col]) + abs(df2[col]) + abs(df3[col]) + abs(df4[col]) + 
                          abs(df5[col]) + abs(df6[col]) + abs(df7[col]) + abs(df8[col]) + abs(df9[col]) + 
                         abs(df10[col]) + abs(df11[col]) + abs(df12[col]) + abs(df13[col]) + abs(df14[col]) + 
                          abs(df15[col]) + abs(df16[col]) + abs(df17[col]) + abs(df18[col]) + abs(df19[col])) / 20

    # Process categorical columns
    for col in df1.columns[35:]:
        output_df[col] = pd.concat([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
                                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=1).median(axis=1)
        
        # median = stats.median([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
        #                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=0).median[0]
        # output_df[col] = median

    # Write the output dataframe to a CSV file
    output_df.to_csv('output_NCa_PCa.csv', index=False)

    # re-order cols in output_df
    output_df_re = output_df[['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 
                              'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 
                              'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG', 'CACHEXSTAGE0VIG']].copy()

    return output_df_re





In [8]:
# calculate proportion of missingness in ndata_noR

# total num of NaN in the ndata_noR
total_nan_count = X_train.isna().sum().sum()

# total num of cells in ndata_noR
total_cells = X_train.size

# proportion of NaN
NaN_proportion = total_nan_count / total_cells

print('NaN_proportions:',NaN_proportion)


NaN_proportions: 0.015444015444015444


In [9]:
# 1.35% missingness

Xy_NCa_PCa_md = midas_impl(Xy_NCa_PCa, col_to_scale, 200)

Size index: [38, 2]

Computation graph constructed

Metal device set to: Apple M1 Pro
Model initialised

Epoch: 0 , loss: 84.32589467366536
Epoch: 1 , loss: 80.94561767578125
Epoch: 2 , loss: 73.88075510660808
Epoch: 3 , loss: 74.70592244466145
Epoch: 4 , loss: 73.52227528889973
Epoch: 5 , loss: 74.93708038330078
Epoch: 6 , loss: 85.7688471476237
Epoch: 7 , loss: 74.46094767252605
Epoch: 8 , loss: 75.50558725992839
Epoch: 9 , loss: 88.89719136555989
Epoch: 10 , loss: 118.66807047526042
Epoch: 11 , loss: 213.6463623046875
Epoch: 12 , loss: 159.2205810546875
Epoch: 13 , loss: 82.68029530843098
Epoch: 14 , loss: 72.72224680582683
Epoch: 15 , loss: 114.99196879069011
Epoch: 16 , loss: 102.03179168701172
Epoch: 17 , loss: 72.3180669148763
Epoch: 18 , loss: 297.565912882487
Epoch: 19 , loss: 105.38908386230469
Epoch: 20 , loss: 185.3743413289388
Epoch: 21 , loss: 77.10263570149739
Epoch: 22 , loss: 121.54788716634114
Epoch: 23 , loss: 79.99268849690755
Epoch: 24 , loss: 64.71712748209636
Epo

In [10]:
Xy_NCa_PCa_md

,ENA.78,IFN.y,IL.10,IL.6,IL.8,MCP.1,MDC,MIP.1a,TNF.a,C.peptide,G.CSF,IL.22,Insulin,Leptin,MIP.3a,GRO.a,HGF,MMP.2,Adiponectin,CRP,GDF.15,TIMP.1,TGF.B2,TGF.B1,PPAR.y,HIF.1a,Laminin,HbA1c,CA19.9,Glucose,HDL,CCK,LDL,Triglyceride,Albumin,Lumican,ZAG,CACHEXSTAGE0VIG
0,0.916012,0.773864,0.466720,0.438745,0.759182,0.934380,0.935658,0.544842,0.914218,0.901158,0.628692,0.551515,0.768640,0.811422,0.578580,0.850270,0.691718,1.000000,0.887441,0.813027,0.913987,0.906359,0.678160,0.905997,0.482105,0.720050,0.948046,0.905567,0.576164,0.806041,0.883013,0.903541,0.917407,0.646522,1.000000,0.941890,0.932254,0.0
1,0.971395,0.675048,0.158630,0.274562,0.714221,0.838201,0.914333,0.447761,0.605457,0.735082,0.595926,0.416123,0.370198,0.698920,0.421846,0.831512,0.610730,0.896706,0.951320,0.696384,0.771947,0.868417,0.765892,0.950686,0.633290,0.761507,0.868251,1.000000,0.302502,0.757426,0.690563,0.872439,0.852130,0.799707,0.992017,0.893023,0.933726,0.0
2,0.767458,0.563962,0.247242,0.405506,0.601407,0.793859,0.908735,0.512873,0.730034,0.827821,0.458908,0.230775,0.631779,0.839729,0.403834,0.665895,0.612578,0.898815,0.918048,0.726353,0.732192,0.840210,0.779090,0.896131,0.432895,0.698575,0.890141,0.846251,0.971578,0.723235,0.858347,0.764250,0.823028,0.820886,0.981886,0.909395,0.926294,0.0
3,0.843659,0.680724,0.548856,0.501163,0.742832,0.819483,0.924754,0.753950,0.826945,0.981980,0.483916,0.606013,0.914740,0.917280,0.541645,0.786253,0.618417,0.934024,0.946465,0.813461,0.845660,0.892963,0.374739,0.941544,0.517790,0.835278,0.806182,0.857514,0.997719,0.900699,0.938710,0.827422,0.871011,0.812079,0.986790,0.920587,0.938099,0.0
4,0.823597,0.437799,0.241706,0.466195,0.784069,0.824842,0.936134,0.462527,0.786547,0.822344,0.570228,0.344953,0.481480,0.748758,0.450371,0.790204,0.720040,0.946851,0.946150,0.818276,0.861997,0.872046,0.766952,0.921476,0.217695,0.761948,0.875633,0.799898,0.532850,0.778359,0.930197,0.747863,0.886069,0.718185,0.974702,0.944693,0.933486,1.0
5,0.857589,0.651892,0.504551,0.420036,0.742782,0.833994,0.914060,0.509881,0.753772,0.857937,0.533257,0.276327,0.553892,0.778657,0.488851,0.827281,0.650863,0.989362,0.971187,0.692731,0.775382,0.850170,0.866992,0.904974,0.234798,0.565179,0.912093,0.890886,0.487633,0.914067,0.904709,0.792105,0.787856,0.815544,0.973931,0.999781,0.949686,1.0
6,0.741210,0.433991,0.076214,0.258087,0.629969,0.801048,0.901094,0.422081,0.594559,0.754859,0.543084,0.089464,0.412029,0.643633,0.431333,0.700595,0.582678,0.879956,1.000000,0.679046,0.771786,0.855852,0.550166,0.866788,0.345624,0.903705,0.855294,0.934853,0.425660,0.788614,0.877338,0.831072,0.843616,0.811737,0.997037,0.891671,0.933967,1.0
7,0.882740,0.459280,0.045935,0.512290,0.722219,0.845796,0.923254,0.411220,0.618757,0.831002,0.773506,0.344704,0.571074,0.800886,0.434944,0.774280,0.674456,0.873789,0.942873,0.823450,0.885712,0.870891,0.624882,0.935512,0.058474,0.709524,0.783763,0.837697,0.245918,0.799105,0.809718,0.828576,0.821318,0.652735,0.970698,0.925285,0.923828,1.0
8,0.853435,0.648731,0.431962,0.484971,0.783121,0.823659,0.921852,0.580851,0.827650,1.000000,0.619003,0.343739,1.000000,0.844393,0.607107,0.755379,0.645504,0.940128,0.896564,0.787369,0.802425,0.855367,0.478148,0.917457,0.793542,0.981068,0.838545,0.812270,0.815475,0.998132,0.889624,0.797570,0.966994,0.827026,0.980153,0.939009,0.948128,0.0
9,0.828239,0.814621,0.465397,0.675257,0.872671,0.822767,0.886151,0.510833,0.854652,0.810259,0.612417,0.167507,0.544745,0.654425,0.653871,0.789043,0.769121,0.936809,0.916558,0.902168,0.883610,0.904088,0.647286,0.886294,0.210458,0.660826,0.906748,0.906105,0.744953,0.843771,0.929329,0.775526,0.895740,0.589506,0.976502,0.930562,0.934533,1.0


In [11]:
# save to csv file

# 
Xy_NCa_PCa_md.to_csv('data/Xy_NCa_PCa_md.csv', index=False)